Importing Libraries 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import re

Note: User-Agent has been removed from the code. (since its not a generic user agent)

In [3]:
#The url's of the product pages for scrapping into the list for ease usage
url = [ 'https://www.flipkart.com/search?q=furniture&sid=wwe%2Cfc3&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_3_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_1_3_na_na_na&as-pos=1&as-type=RECENT&suggestionId=furniture%7CWardrobes+%26+More&requestId=6468df7f-c9bc-46af-84a1-c58b35c81552&as-backfill=on', 'https://www.flipkart.com/search?q=Computer%20components&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off']
HEADERS = ({'User-Agent':, 'Accept-Language':'en-US, en;q=0.5'})

In [4]:
#The below helps us retrive data from a Product page, kindly uncomment to review the product page

def return_data(link):
    link = link.get('href')
    
    product_link = "https://www.flipkart.com"+ link
    #print(product_link)
    new_webpage = requests.get(product_link, headers=HEADERS)
    if(new_webpage == 503): #error handling
        print('Response 503')
        return ['Error']
    new_soup = BeautifulSoup(new_webpage.content, 'html.parser') #using beautifulsoup to praise html text
    p_name = new_soup.find("span", attrs={'class': 'B_NuCI'}).text.strip() #product name
    i_link = new_soup.find("img", attrs={'class': '_396cs4 _2amPTt _3qGmMb'})['src'] #image link
    price = new_soup.find('div', attrs={'class': '_30jeq3 _16Jk6d'}).text #price
    price_cleaned = re.sub(r'[^\d.]', '', price)
    price = float(price_cleaned) #removing unnecessary char's from price
    rating = new_soup.find('div', attrs={'class': '_3LWZlK'}).text #rating of the product
    des = new_soup.find('div', attrs={'class': '_1mXcCf RmoJUa'}).text #description of product
    review_title = new_soup.find('p', attrs={'class':'_2-N8zT'}).text #review title
    review = new_soup.find('div', attrs = {'class': 't-ZTKy'}).text #review
    read_more_index = review.find("READ MORE") # if "READ MORE" is found, removing it along with any trailing space
    if read_more_index != -1:
        review = review[:read_more_index].strip()

    info = [p_name, i_link, price, rating, des, review_title, review]
    
    return info #returns a list

In [5]:
#Get data from all the pages 
#we can extract data from multiple pages without manually inputing links of next pages
def data_all_pages(pages, no_of_products):
    if(len(pages) == 0):
        print("No links exist in Pages")
        return []
    category_data = []
    for i in pages:
        sub_page = "https://www.flipkart.com"+ i
        #print(sub_page)
        sub_response = requests.get(sub_page, headers=HEADERS)
        if(sub_response.status_code == 503):
            print("503 Error encountered. Breaking the loop.")
        sub_soup = BeautifulSoup(sub_response.content, 'html.parser')
        product_links = sub_soup.find_all("a", attrs={'class' : 's1Q9rs'})
        for link in product_links:
            try:
                category_data.append(return_data(link))
            except AttributeError as e: #if some of the data is empty we will get an error here
                #error handling
                #print(f"In {link} Attribute error occurred: {e}")
                continue
            if(len(category_data) == no_of_products): 
                return category_data
    return category_data

In [6]:
#inputs: list of URL's, file_names to save the file, Number of products to extract
#outputs: Generates .csv files

def get_data(urls, file_names, no_of_products):
    if(len(urls) != len(file_names)):
        print("Size of urls and file_names doesnt match")
        return
    i_fn = 0 #to iterate over the file names
    for url in urls: 
        #print(url)
        response = requests.get(url, headers=HEADERS) #request to the server
        if(response.status_code == 503): #error handling
            print("503 Error encountered. Breaking the loop.")
        soup = BeautifulSoup(response.content, 'html.parser')
        div = soup.find('div', class_='_2MImiq')
        a_tags = div.find_all('a')
        a_tags.pop()
        all_pages = [tag['href'] for tag in a_tags if 'href' in tag.attrs]
        data = data_all_pages(all_pages, no_of_products)
        with open(file_names[i_fn], 'w', newline='', encoding='utf-8') as csvfile:
            print(f"Writing file to {file_names[i_fn]}") #for checking progress in case of multiple link
            i_fn+=1
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Product Name', 'Image Link', 'Price', 'Rating', 'Description', 'Review Title', 'Review'])
            for item in data:
                csvwriter.writerow(item)

In [7]:
len(url)

2

In [8]:
get_data(url, ['furniture.csv', 'computer_componenets.csv'], 20)

Writing file to furniture.csv
Writing file to computer_componenets.csv
